In [ ]:
import numpy as np
import torch
from torch import nn
from torch.nn import functional as F
import pandas as pd
from tqdm import tqdm
from transformers import logging
logging.set_verbosity_error()
from transformers import BertModel, BertTokenizer, BertForMaskedLM, pipeline
from sentence_transformers import SentenceTransformer, util

print(torch.cuda.is_available())
print(torch.__version__)

import os
from pathlib import Path
# os.chdir(Path(os.environ["ALTHOME"]) / "Master-thesis")

In [ ]:
l = ["Ä","ä","Ö","ö","Ü","ü","ß"]
replacements = {key: str(bytes(key, encoding="utf-8"), encoding="latin-1") for key in l}

def replace(s):
    for k,v in replacements.items():
        s = s.replace(v, k)
    return s

if False:
    with open("data/SimpleGerman/spacy_easy.txt") as fp:
        text = fp.read()
    text = replace(text)
    with open("data/SimpleGerman/fixed_easy.txt", "w") as fp:
        fp.write(text)
    with open("data/SimpleGerman/spacy_normal.txt") as fp:
        text = fp.read()
    text = replace(text)
    with open("data/SimpleGerman/fixed_normal.txt", "w") as fp:
        fp.write(text)

In [ ]:
model_name = "dbmdz/bert-base-german-cased"
model_name = "deepset/gbert-base"
sentence = "Dies ist ein Satz"
model_name = "deepset/gbert-base"

pipe  = pipeline(model=model_name, tokenizer=model_name, task="feature-extraction")
model_encode = BertModel.from_pretrained(model_name)
model_decode = BertModel.from_pretrained(model_name, is_decoder=True, add_cross_attention=True)
model_LM = BertForMaskedLM.from_pretrained(model_name)
tokenizer = BertTokenizer.from_pretrained(model_name)

tokens = tokenizer.encode(sentence, return_tensors="pt")
output = model_encode.forward(tokens, output_hidden_states=True)
output_pipe = pipe(sentence)

a = output.hidden_states[-1].detach().numpy() # same as pipeline feature extraction
c = output.last_hidden_state.detach().numpy()
b = np.array(output_pipe)

print((a == b).all())
print((a == c).all())
print(f"{type(a)} - {a.shape}")
print(f"{type(b)} - {b.shape}")
print(f"{type(c)} - {c.shape}")
tokenizer = BertTokenizer.from_pretrained(model_name)
tokens = tokenizer.encode(sentence, return_tensors="pt")
tokens2 = tokenizer.encode(sentence)
output_encode = model_encode.forward(tokens)
output_LM = model_LM(tokens, output_hidden_states=True)

print(type(tokens))
print(type(tokens2))

print(output_encode.last_hidden_state.size())
print(output_encode.pooler_output.size())
print(output_encode.keys())
print(output_LM.keys())
print((output_encode.last_hidden_state == output_LM.hidden_states[-1]))

In [ ]:
sentences = ["Dies ist ein Test, welcher ganz ok läuft.","Also geht es jetzt los.","'Mein Vater', sagt er."]
model_name="deepset/gbert-base"

vocab_size= len(tokenizer.get_vocab())
d_model = 256
embedder = nn.Embedding(vocab_size, d_model)
tokens = torch.IntTensor([tokenizer.encode(sent, padding="max_length") for sent in sentences])
max_sent_length= tokenizer.model_max_length

tgt_mask = torch.triu(torch.ones(max_sent_length-1, max_sent_length-1) * float('-inf'), diagonal=1)

tgt = tokens[:,:-1].clone()
tgt[tgt==103] = 0
tgt_key_padding_mask = (tgt == 0)
src_key_padding_mask = (tokens == 0)

tgt = embedder(tgt)tgt_key_padding_mask
src = embedder(tokens)
tgt_result = embedder(tokens[:,1:])


encoder_layer = nn.TransformerEncoderLayer(d_model=d_model, nhead=8,batch_first=True)
transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=6)
decoder_layer = nn.TransformerDecoderLayer(d_model=d_model, nhead=8,batch_first=True)
transformer_decoder = nn.TransformerDecoder(decoder_layer, num_layers=6)

memory = transformer_encoder(src,src_key_padding_mask=src_key_padding_mask)
output = transformer_decoder(tgt,memory,tgt_mask=tgt_mask,tgt_key_padding_mask=tgt_key_padding_mask)


print(tokenizer.decode([0,1,100,101,102,103,104]))

In [ ]:
# Example of target with class indices
input = torch.randn(3, 5, requires_grad=True)
target = torch.randint(5, (3,), dtype=torch.int64)
loss = F.cross_entropy(input, target)
print(input)
print(target)

# Example of target with class probabilities
input = torch.randn(3, 5, requires_grad=True)
target = torch.randn(3, 5).softmax(dim=1)
loss = F.cross_entropy(input, target)
print(input)
print(target)
print(loss)
loss = F.cross_entropy(F.log_softmax(input, dim=-1), target)
print(loss)